# 2018 Later 5 Years

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2025-01-13  


## 1. Import packages

In [1]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../../')
sys.path.insert(1,'../../../')
from config import conf as cfg
from tools import  bioFunctionLib as bfl
import rxnrecer as production
from modules import commonfunction as cmfunc
from tqdm import tqdm
import re
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
from tkinter import _flatten
import json
from tools import btools
from evaluation import evTools
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2,venn3

FIRST_TIME_RUN = False # For the initial run, please set this flag to True. This will allow the program to download data from UniProt and RHEA, which may take longer depending on your internet speed.

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. TestSet for Newly 5 Years

In [2]:
test_set = pd.read_feather(cfg.FILE_DS_TEST)
test_set = test_set[['uniprot_id', 'reaction_id', 'ec_number', 'label']].rename(columns={'reaction_id': 'rxn_groundtruth', 'ec_number': 'ec_groundtruth', 'label': 'rxn_groundtruth_label'})
test_set.head(3)

,uniprot_id,rxn_groundtruth,ec_groundtruth,rxn_groundtruth_label
0,A9JLI2,-,-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,A9JLI3,-,-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,A9JLI5,-,-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## 3. Load Results

In [5]:
res_ec  = pd.read_feather(f'{cfg.CASE_2018LATER}res/res_methods_ec.feather')
res_direct = pd.read_feather(f'{cfg.CASE_2018LATER}res/res_methods_direct.feather')

In [8]:
pd.set_option('display.max_columns', None)
res_direct.head(3)

,uniprot_id,rxn_groundtruth,enz_groundtruth,lb_rxn_groundtruth,rxn_blast,lb_rxn_blast,rxn_eu_esm,enz_eu_esm,rxn_cos_esm,enz_cos_esm,lb_rxn_eu_esm,lb_rxn_cos_esm,rxn_eu_unirep,enz_eu_unirep,rxn_cos_unirep,enz_cos_unirep,lb_rxn_eu_unirep,lb_rxn_cos_unirep,rxn_eu_t5,enz_eu_t5,rxn_cos_t5,enz_cos_t5,lb_rxn_eu_t5,lb_rxn_cos_t5,enz_RXNRECer,rxn_RXNRECer,lb_rxn_RXNRECer
0,A9JLI2,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,A9JLI3,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,A9JLI5,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
res_ec.head(3)

,uniprot_id,rxn_groundtruth,isenzyme_groundtruth,ec_groundtruth,ec_specific_level,lb_rxn_groundtruth,ec_ecblast,reaction_ecblast,ec_deepec,reaction_deepec,ec_clean,reaction_clean,ec_ecrecer,reaction_ecrecer,ec_ecpred,reaction_ecpred,ec_catfam,reaction_catfam,ec_priam,reaction_priam,lb_rxn_ecblast,lb_rxn_deepec,lb_rxn_clean,lb_rxn_ecrecer,lb_rxn_ecpred,lb_rxn_catfam,lb_rxn_priam
0,A9JLI2,-,False,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,-,NO-PREDICTION,NO-PREDICTION,3.2.2.6;1.4.3.2;4.2.3.81,RHEA:31427;RHEA:16301;RHEA:13781,-,-,-,-,-,-,NO-PREDICTION,NO-PREDICTION,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,A9JLI3,-,False,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,-,NO-PREDICTION,NO-PREDICTION,4.6.1.18,EC-WITHOUT-REACTION,-,-,-,-,-,-,1.14.11.51;2.3.2.27,RHEA:49524,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,A9JLI5,-,False,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,-,NO-PREDICTION,NO-PREDICTION,1.4.3.2,RHEA:13781,-,-,-,-,-,-,6.5.1.3,EC-WITHOUT-REACTION,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## 4. 整理表格

In [15]:
res = res_direct.merge(res_ec, on='uniprot_id', how='left')
res.head(2)

,uniprot_id,rxn_groundtruth_x,enz_groundtruth,lb_rxn_groundtruth_x,rxn_blast,lb_rxn_blast,rxn_eu_esm,enz_eu_esm,rxn_cos_esm,enz_cos_esm,lb_rxn_eu_esm,lb_rxn_cos_esm,rxn_eu_unirep,enz_eu_unirep,rxn_cos_unirep,enz_cos_unirep,lb_rxn_eu_unirep,lb_rxn_cos_unirep,rxn_eu_t5,enz_eu_t5,rxn_cos_t5,enz_cos_t5,lb_rxn_eu_t5,lb_rxn_cos_t5,enz_RXNRECer,rxn_RXNRECer,lb_rxn_RXNRECer,rxn_groundtruth_y,isenzyme_groundtruth,ec_groundtruth,ec_specific_level,lb_rxn_groundtruth_y,ec_ecblast,reaction_ecblast,ec_deepec,reaction_deepec,ec_clean,reaction_clean,ec_ecrecer,reaction_ecrecer,ec_ecpred,reaction_ecpred,ec_catfam,reaction_catfam,ec_priam,reaction_priam,lb_rxn_ecblast,lb_rxn_deepec,lb_rxn_clean,lb_rxn_ecrecer,lb_rxn_ecpred,lb_rxn_catfam,lb_rxn_priam
0,A9JLI2,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,False,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,-,NO-PREDICTION,NO-PREDICTION,3.2.2.6;1.4.3.2;4.2.3.81,RHEA:31427;RHEA:16301;RHEA:13781,-,-,-,-,-,-,NO-PREDICTION,NO-PREDICTION,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,A9JLI3,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,0,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,-,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,False,-,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-,-,NO-PREDICTION,NO-PREDICTION,4.6.1.18,EC-WITHOUT-REACTION,-,-,-,-,-,-,1.14.11.51;2.3.2.27,RHEA:49524,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [38]:
res = res[['uniprot_id', 'rxn_groundtruth_x',
     'reaction_ecblast','reaction_deepec','reaction_clean','reaction_ecrecer','reaction_catfam','reaction_priam',
     'rxn_blast','rxn_cos_esm','rxn_cos_unirep','rxn_cos_t5','rxn_RXNRECer'
     ]].rename(columns={'rxn_groundtruth_x': 'rxn_groundtruth',
                        'reaction_ecblast':'MSA-via-EC',
                        'reaction_deepec':'DeepEC',
                        'reaction_clean':'CLEAN',
                        'reaction_ecrecer':'ECRECer',
                        'reaction_catfam':'CatFam',
                        'reaction_priam':'PRIAM',
                        'rxn_blast':'MSA-via-RXN',
                        'rxn_cos_esm':'ESM',
                        'rxn_cos_unirep':'UniRep',
                        'rxn_cos_t5':'T5',
                        'rxn_RXNRECer':'RXNRECer'
                        })
res.head(2)

,uniprot_id,rxn_groundtruth,MSA-via-EC,DeepEC,CLEAN,ECRECer,CatFam,PRIAM,MSA-via-RXN,ESM,UniRep,T5,RXNRECer
0,A9JLI2,-,-,NO-PREDICTION,RHEA:31427;RHEA:16301;RHEA:13781,-,-,NO-PREDICTION,-,-,-,-,-
1,A9JLI3,-,-,NO-PREDICTION,EC-WITHOUT-REACTION,-,-,RHEA:49524,-,-,-,-,-


In [37]:
methods = ['MSA-via-EC', 'DeepEC', 'CLEAN', 'ECRECer', 'CatFam', 'PRIAM', 'MSA-via-RXN', 'ESM', 'UniRep', 'T5', 'RXNRECer']

In [39]:
for item in methods:
    res[f'pred_true_{item}'] = res.apply(
        lambda x: set(x.rxn_groundtruth.split(cfg.SPLITER)).issubset(set(x[f'{item}'].split(cfg.SPLITER))),
        axis=1
    )

In [53]:
for item in methods:
    correct = res[res[f'pred_true_{item}']].shape[0]
    accuracy = round(correct / len(res), 6)
    print(f'{item:12s}: {accuracy}')


MSA-via-EC  : 0.43744
DeepEC      : 0.036108
CLEAN       : 0.10899
ECRECer     : 0.695154
CatFam      : 0.771143
PRIAM       : 0.118461
MSA-via-RXN : 0.516907
ESM         : 0.739327
UniRep      : 0.698705
T5          : 0.760858
RXNRECer    : 0.957085


In [54]:
# savedf.to_excel('res/res_case2018Later_250115.xlsx', index=None)

In [59]:
def count_num(rxnstr):
    if rxnstr =='-':
        return 0
    
    rxns = rxnstr.split(cfg.SPLITER)
    if '-' in rxns:
        return len(rxns) -1
    else:
        return len(rxns)

In [63]:
res['rxn_num_groundtruth'] = res.rxn_groundtruth.apply(lambda x: count_num(x))
res['rxn_num_rxnrecer'] = res.RXNRECer.apply(lambda x: count_num(x))

In [66]:
res[(res.rxn_num_rxnrecer > res.rxn_num_groundtruth) & (res.rxn_groundtruth!='-')]

,uniprot_id,rxn_groundtruth,MSA-via-EC,DeepEC,CLEAN,ECRECer,CatFam,PRIAM,MSA-via-RXN,ESM,UniRep,T5,RXNRECer,pred_true_MSA-via-EC,pred_true_DeepEC,pred_true_CLEAN,pred_true_ECRECer,pred_true_CatFam,pred_true_PRIAM,pred_true_MSA-via-RXN,pred_true_ESM,pred_true_UniRep,pred_true_T5,pred_true_RXNRECer,rxn_num_groundtruth,rxn_num_rxnrecer
16,Q9SNN8,RHEA:21744,RHEA:21744,RHEA:21744,RHEA:21744;RHEA:15093;RHEA:22104,RHEA:21744,-,RHEA:14089;RHEA:24248;RHEA:23744;RHEA:19453;RH...,RHEA:21744,RHEA:22104,RHEA:19453,-,RHEA:22104;-;RHEA:21744,True,True,True,True,False,True,True,False,False,False,True,1,2
18,M5AWY0,RHEA:34415,EC-WITHOUT-REACTION,EC-WITHOUT-REACTION,RHEA:34415,RHEA:34415,-,RHEA:48108,RHEA:34415,RHEA:34415,RHEA:34415,RHEA:34415,RHEA:34415;-;RHEA:56596,False,False,True,True,False,False,True,True,True,True,True,1,2
20,Q88FI7,RHEA:12601;RHEA:10212,RHEA:13993;RHEA:23352,NO-PREDICTION,RHEA:10212,RHEA:12268,RHEA:23352,RHEA:14813;RHEA:16953;RHEA:14265;RHEA:14077;RH...,RHEA:23352;RHEA:10212,RHEA:18049,RHEA:18049,RHEA:23352,RHEA:18049;RHEA:23352;RHEA:10212;RHEA:12268,False,False,False,False,False,False,False,False,False,False,False,2,4
21,A5CAL1,RHEA:16369;RHEA:18657;RHEA:18229;RHEA:10992,RHEA:57408,RHEA:57408,RHEA:57704;RHEA:18229;RHEA:57408;RHEA:10780;RH...,RHEA:10992;RHEA:16369;RHEA:18229,-,RHEA:15985;RHEA:18657;RHEA:12641;RHEA:10780;RH...,RHEA:10780;RHEA:10776;RHEA:57704;RHEA:57408,RHEA:10992;RHEA:17905;RHEA:18657,RHEA:10992;RHEA:17905;RHEA:18657,RHEA:10780;RHEA:10776;RHEA:57704;RHEA:57408,RHEA:10992;RHEA:17905;RHEA:18657;RHEA:10780;RH...,False,False,False,False,False,True,False,False,False,False,False,4,7
29,A7B3K3,RHEA:47496,RHEA:52584,RHEA:17397,RHEA:47496,RHEA:47496,-,RHEA:35479;RHEA:11728;RHEA:12184;RHEA:45232;RH...,RHEA:55380;RHEA:52588;RHEA:52584;RHEA:52592;RH...,RHEA:55380;RHEA:52588;RHEA:52584;RHEA:52592;RH...,RHEA:55380;RHEA:52588;RHEA:52584;RHEA:52592;RH...,RHEA:11728;RHEA:10044,RHEA:55380;RHEA:52596;RHEA:52584;RHEA:52592;RH...,False,False,True,True,False,False,False,False,False,False,True,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13225,Q9RV46,RHEA:64988,NO-PREDICTION,NO-PREDICTION,RHEA:67484;RHEA:35211;RHEA:35207;RHEA:31575,EC-WITHOUT-REACTION,-,RHEA:31575;RHEA:10412,NO-PREDICTION,RHEA:60876;RHEA:11800;RHEA:48868,RHEA:21732,RHEA:50044;RHEA:64988;RHEA:64992;RHEA:49976;RH...,RHEA:67592;RHEA:60876;RHEA:11800;RHEA:48868;RH...,False,False,False,False,False,False,False,False,False,True,True,1,15
13401,O77334,RHEA:20629;RHEA:10684,RHEA:47004;RHEA:10684;RHEA:20629,RHEA:10684,RHEA:10596,RHEA:20629;RHEA:47004;RHEA:10684,-,RHEA:49524;RHEA:10684;RHEA:16065;RHEA:50664;RH...,RHEA:10684;RHEA:20629;RHEA:47004,RHEA:43636,-,RHEA:36899,RHEA:10684;RHEA:43636;RHEA:36899;RHEA:20629;RH...,True,False,False,True,False,True,True,False,False,False,True,2,5
13465,Q7BQ71,RHEA:61704,-,NO-PREDICTION,RHEA:61704,RHEA:61704,-,RHEA:11584;RHEA:20201;RHEA:41640;RHEA:19229;RH...,-,RHEA:11584,RHEA:55132,RHEA:55132,RHEA:11584;RHEA:55132;RHEA:61704;-;RHEA:45624,False,False,True,True,False,False,False,False,False,False,True,1,4
13480,E9P860,RHEA:13065,-,EC-WITHOUT-REACTION,EC-WITHOUT-REACTION,RHEA:13065,-,RHEA:13065,-,-,-,RHEA:22508;RHEA:22508,RHEA:13065;RHEA:17989;RHEA:46608;RHEA:22508;-,False,False,False,True,False,True,False,False,False,False,True,1,4


In [67]:
res.to_excel('res/res_ensemble_2018later_all_250213.xlsx', index=False)